In [2]:
import re
import sys
import os
import math
from pyautocad import Autocad, APoint, aDouble
from math import *
import pandas as pd
import win32com.client
sys.path.insert(1,os.getcwd())
from utils import eq
from utils import useful_functions as uf
from entities import entities
from utils.autocad import autocad_functions

acad = Autocad(create_if_not_exists=True)
# pipes_table = pd.read_excel('data\\outputs\\acad-pipelines.xlsx',sheet_name='Pipes')
# pipes_table.set_index('pipe #')
# pumps_table = pd.read_excel('data\\outputs\\acad-pipelines.xlsx',sheet_name='Pumps')
# pumps_table.set_index('pump #')

# def float_tuple_from_str(string_tuple):
#     '''
#     string_tuple = '(1.0,2.0,3.0)'
#     float_tuple = (1.0,2.0,3.0)
#     '''
#     string_tuple.strip('()').split(',')
#     float_tuple = tuple(map(float, string_tuple.strip('()').split(',')))
#     return float_tuple

# def midpoint_betwen_to_points(p1,p2):
#     p3 = tuple([(p1[i]+p2[i])/2 for i,cor in enumerate(p1)])
#     return p3
    
# pipes_table

In [22]:
text = 'Please insert input: '
a = acad.doc.Utility.Prompt.func
# b = input(a)
a

AttributeError: 'function' object has no attribute 'func'

In [ ]:
help(acad.doc.Utility.Prompt('a'))

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      self != 0
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [ ]:
steel_pipes_table = pd.read_excel('pipes.xlsx',sheet_name='Steel')
# steel_pipes_table = steel_pipes_table.groupby('OD')
steel_pipes_table.head()
# steel_pipes_table.info()
# type(steel_pipes_table[2])

,ND,OD,allowed pressure for GR-B,allowed pressure for x-42,wall thickness,Cement thickness,Water content,Id,Flow area,Support distance
0,"""",mm,bar,bar,mm,mm,kg,m,m^2,m
1,3,88.9,106,129,3.97,5,4.2,0.073,0.0042,6
2,3,88.9,129,155,4.76,5,4,0.071,0.004,6
3,3,88.9,151,181,5.56,5,3.8,0.07,0.0038,7
4,4,114.3,84,101,3.97,5,7.6,0.098,0.0076,7


In [ ]:
steel_pipes_table.groupby('OD').unstack().head()

AttributeError: 'DataFrameGroupBy' object has no attribute 'unstack'

In [ ]:
steel_pipes_table.iloc[(float(steel_pipes_table['Id'])-422).abs().argsort()[:-1]]

TypeError: cannot convert the series to <class 'float'>

In [ ]:
pd.to_numeric(steel_pipes_table['Id'],errors='coerce',downcast='float')
steel_pipes_table['Id'][0]=0

In [ ]:
type(steel_pipes_table['Id'][2])

float

In [ ]:
def find_neighbours(value, df, colname):
    exactmatch = df[df[colname] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        print(type(df[colname][1]))
        print(type(value))
        lowerneighbour_ind = df[df[colname] < value][colname].idxmax()
        return [lowerneighbour_ind]
find_neighbours(0.422, steel_pipes_table, 'Id')

<class 'float'>
<class 'float'>


TypeError: reduction operation 'argmax' not allowed for this dtype

# find dia via velocity

In [ ]:
result_index = steel_pipes_table[steel_pipes_table['Id']-0.442 > 0]['Id'].min()
result_index#['Id'].min()


0.467

In [ ]:
result_index = steel_pipes_table[steel_pipes_table['Id']-0.442 > 0]['Id'].min()
steel_pipes_table.iloc(steel_pipes_table[steel_pipes_table['Id']==result_index]['ND'])

TypeError: unhashable type: 'Series'

# autocad stuff

In [ ]:
from html import entities
import sys
import os
from pyautocad import Autocad, APoint, aDouble
from math import *
import pandas as pd

sys.path.insert(1,os.getcwd())
from utils import eq
from entities import entities

acad = Autocad(create_if_not_exists=True)


'''
running on the dwg and sorting it's objects:
'''
i = 0
j = 0
channels_table = pd.DataFrame(columns=['pipe #', 'start', 'end', 'pipe type',
    'nominal dia', 'id (mm)', 'length (m)', 'static head (endZ-startZ)', 
    'flow', 'consumption', 'velocity', 'head loss', 'total head loss', 'start with', 'end with'])
pumps_table = pd.DataFrame(columns=['pump #', 'center', 'flow' , 'head','efficiency','start num','power','wetpit min volume','connect to pipe'])

for obj in acad.iter_objects_fast():

    # print("Type of object: " + obj.ObjectName)
    
    if obj.ObjectName == "AcDbLine":
        
        # ### Add only a new pipes!!!  ### comment out becuse it's not tolerate with changes!
        # if pipes_table[(pipes_table['start'] == str(obj.Startpoint)) & (pipes_table['end'] == str(obj.Endpoint))].empty:
        
        if obj.Layer.startswith('Pipe_'):
            # if obj.Startpoint in pipes_table & obj
            pipe_name = 'Pipe '+ str(i+1)
            i += 1
            tmp,pipetype, nd , consumption= obj.Layer.split('_')
            length=obj.Length
            static_head=obj.Endpoint[2]-obj.Startpoint[2]
            pipe = entities.Pipe(pipetype=pipetype,nominal_dia=float(nd),length=length,static_head=static_head)
            pipe.inside_dia = pipe.inside_dia_from_nominal(nd=pipe.nominal_dia)

            data = [pipe_name,obj.Startpoint,obj.Endpoint,pipetype,nd,pipe.inside_dia,pipe.length,
                        pipe.static_head,"",consumption,"",
                        "","","",""]
            channels_table.loc[len(channels_table)] = data

    # if obj.ObjectName == 'AcDbBlockReference':
    #     print(obj.Rotation*57.2957795)
    #     for obj1 in obj.iter_objects_fast():
    #         print(1)

    # i = 0
    if (obj.Layer.startswith('Pump_')) & (obj.ObjectName == "AcDbCircle"):
        pump_name = 'Pump '+ str(j+1)
        j += 1
        tmp,flow = obj.Layer.split('_')
        pump = entities.Pump(rated_flow=flow)
        efficiency = 0.75 # Will change in the futere 
        # wetpit = pump.min_wet_pit(flow,8)
        data = [pump_name,obj.Center,flow,0,efficiency,'','','','']
        pumps_table.loc[len(pumps_table)] = data
        #'pump #', 'center', 'flow' , 'head','efficiency','start num','power','wetpit min volume','connect to pipe'
# pipes_table['start']

channels_table.set_index(channels_table['pipe #'],inplace=True)
pumps_table.set_index(pumps_table['pump #'],inplace=True)

In [ ]:
for i in range (len(channels_table)):
    start_with_pipe_name = ''
    end_with_pipe_name = ''
#     print(f'i={i}')
    i_start_at = channels_table['start'].iloc[i]
    i_end_at = channels_table['end'].iloc[i]
    # print(i_start_at[0])
    # if pipes_table[pipes_table['start']
    if not (channels_table[channels_table['end'] == i_start_at].empty):
#         print(f'pipe {i+1} is connected at point {a1}!')
        i_start_with = channels_table[channels_table['end'] == i_start_at]
        start_with_pipe_name = ', '.join(i_start_with['pipe #'])
        channels_table['start with'].iloc[i] = start_with_pipe_name
    else:
        if not (pumps_table[pumps_table['center'] == i_start_at].empty):
            i_start_with = pumps_table[pumps_table['center'] == i_start_at]
            # print (i_start_with)
            start_with_pump_name = ', '.join(i_start_with['pump #'])
            print(f'pump name:{start_with_pump_name}')
            channels_table['start with'].iloc[i] = start_with_pump_name
            print(channels_table['pipe #'].iloc[i])
            pumps_table['connect to pipe'][start_with_pump_name] = channels_table['pipe #'].iloc[i]


    if not (channels_table[channels_table['start'] == i_end_at].empty):
#         print(f'pipe {i+1} is delivered to point {a2}!')
        i_end_with = channels_table[channels_table['start'] == i_end_at]
        end_with_pipe_name = ', '.join(i_end_with['pipe #'])
        channels_table['end with'].iloc[i] = end_with_pipe_name
    
pumps_table    
# pipes_table['start with']

pump name:Pump 1
Pipe 1


C:\Users\avner\AppData\Local\Temp\ipykernel_36336\2712501588.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['start with'].iloc[i] = start_with_pump_name
C:\Users\avner\AppData\Local\Temp\ipykernel_36336\2712501588.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pumps_table['connect to pipe'][start_with_pump_name] = pipes_table['pipe #'].iloc[i]
C:\Users\avner\AppData\Local\Temp\ipykernel_36336\2712501588.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

,pump #,center,flow,head,efficiency,start num,power,wetpit min volume,connect to pipe
pump #,,,,,,,,,
Pump 1,Pump 1,"(21.5668043743036, 11.188186431626036, -5.0)",1500,0,0.75,,,,Pipe 1


In [ ]:
channels_table
pumps_table

,pump #,center,flow,head,efficiency,start num,power,wetpit min volume,connect to pipe
pump #,,,,,,,,,


In [ ]:
def min_wet_pit(flow_rate,starts_num):
        
        cycle_time = 60/starts_num/60
        print(cycle_time)
        return (flow_rate*cycle_time/4)

min_wet_pit(1500,8)

0.125


46.875

In [ ]:

def power( flow_rate,head,eff=1):

    # '''
    # if the flow value is less than 4 
    # it assuming that the units are m^3/second. 
    # if it's more then that the units are in m^3/hour
    # '''

    # if flow_rate > 4:
    #     unit_convert = 3600
    # else:
    #     unit_convert = 3.6
    # print (unit_convert)
    # if eff > 1 :
    #     eff /= 1000
    # print(eff)
    return (((flow_rate/3.6)*head)/102)/eff

power(1500,60,0.75)

326.797385620915

# add text to autocad file

In [ ]:
pumps_table['center'][0] +=3


AttributeError: 'tuple' object has no attribute 'x'

In [ ]:
p1 = APoint(pumps_table['center'][0])
p1.y = 30
flow = str(pumps_table['pump #'][0])
text = acad.model.Addtext(flow,p1,2.5)

In [ ]:
for i in range(len(pumps_table)):
        p1 = APoint(pumps_table['center'][i])
        p1.y = 30
        pump_name = str(pumps_table['pump #'][i])
        text = acad.model.Addtext(pump_name,p1,2.5)
        
        p1.y -= 10
        flow = f"{pumps_table['flow'][i]} m^3/hr"
        text = acad.model.Addtext(flow,p1,2.5)

        p1.y -= 10
        head = f"{pumps_table['head'][i]} m"
        text = acad.model.Addtext(head,p1,2.5)

In [ ]:
p1 = APoint(channels_table['start'][0])
p2 = APoint(channels_table['end'][0])
p3 = (p2+p1)/2
p3.y = 80
vertex_text1 = f'Vertex number: {0+1}'
vertex_text2 = f'nZ={p1.z}'
text = acad.model.Addtext(vertex_text1,p1,2.5)
p1.y -= 10
text = acad.model.Addtext(vertex_text2,p1,2.5)

In [ ]:
channels_table = pd.read_excel('data\\outputs\\acad-pipelines.xlsx',sheet_name='Pipes')
pumps_table = pd.read_excel('data\\outputs\\acad-pipelines.xlsx',sheet_name='Pumps')

pumps_table

,pump #,center,flow,head,efficiency,start num,power,wetpit min volume,connect to pipe
0,Pump 1,"(21.5668043743036, 11.188186431626036, -5.0)",1500,49.145897,0.75,8,267.679175,46.875,Pipe 1


(21.5668043743036, 11.188186431626036, -5.0)


TypeError: must be real number, not str

In [ ]:
# a = [float(i) for i in pumps_table['center'][0]]
a = tuple(map(float, pumps_table['center'][0].strip('()').split(',')))
a

(21.5668043743036, 11.188186431626036, -5.0)

In [ ]:
def float_tuple_from_str(string_tuple):
    '''
    string_tuple = '(1.0,2.0,3.0)'
    float_tuple = (1.0,2.0,3.0)
    '''
    string_tuple.strip('()').split(',')
    float_tuple = tuple(map(float, string_tuple.strip('()').split(',')))
    return float_tuple

def midpoint_betwen_to_points(p1,p2):
    p3 = tuple([(p1[i]+p2[i])/2 for i,cor in enumerate(p1)])
    return p3

In [ ]:
### Pump writing info:
for i in range(len(pumps_table)):
               
        # print(float_tuple_from_excel( pumps_table['center'][i]))

        p1 = APoint(float_tuple_from_str(pumps_table['center'][i]))
        p1.y = 30
        pump_name = str(pumps_table['pump #'][i])
        text = acad.model.Addtext(pump_name,p1,2.5)
        
        p1.y -= 4
        flow = f"flow: {pumps_table['flow'][i]} m^3/hr"
        text = acad.model.Addtext(flow,p1,2.5)

        p1.y -= 4
        head = f"head: {pumps_table['head'][i].round(2)} m"
        text = acad.model.Addtext(head,p1,2.5)

### Pipe writing info:

for i in range(len(channels_table)):
    
    p1 = APoint(float_tuple_from_str(channels_table['start'][i]))

    vertex_text1 = f'Vertex number: {i+1}'
    vertex_text2 = f'Z={p1.z}'
    text = acad.model.Addtext(vertex_text1,p1,2.5)
    p1.y -= 4
    text = acad.model.Addtext(vertex_text2,p1,2.5)

    p2 = APoint(float_tuple_from_str(channels_table['end'][i]))
    p3 = APoint(midpoint_betwen_to_points(p2,p1))
    
    p3.y += 80

    pipe_name = str(channels_table['pipe #'][i])
    text = acad.model.Addtext(pipe_name,p3,2.5)
    
    p3.y -= 4
    
    if channels_table['pipe type'][i] == 'Steel':
        units = '"'
    else:
        units = 'mm'

    pipe_type = f"{channels_table['pipe type'][i]} %%C {channels_table['nominal dia'][i]} {units}"
    text = acad.model.Addtext(pipe_type,p3,2.5)
    
    p3.y -= 4
    length = f"length: {channels_table['length (m)'][i]} m"
    text = acad.model.Addtext(flow,p3,2.5)

    p3.y -= 4
    flow = f"flow: {channels_table['flow'][i]} m^3/hr"
    text = acad.model.Addtext(flow,p3,2.5)

    p3.y -= 4
    flow = f"Total head loss: {channels_table['total head loss'][i].round(2)} m"
    text = acad.model.Addtext(flow,p3,2.5)

    if i != (len(channels_table)-1):
        print (i, '   ', len(channels_table))
        

0     4
1     4
2     4


In [ ]:
# p1 = APoint(float_tuple_from_str(pipes_table['start'][i]))
# p2 = APoint(float_tuple_from_str(pipes_table['end'][i]))
p1 = float_tuple_from_str(channels_table['start'][0])
p2 = float_tuple_from_str(channels_table['end'][0])

print(p1,p2)

# p3 = (p2+p1)/2
p3 = tuple([(p1[i]+p2[i])/2 for i,cor in enumerate(p1)])
p3


(21.5668043743036, 11.188186431626036, -5.0) (517.0784291680882, 78.03300443738927, 10.0)


(269.3226167711959, 44.61059543450765, 2.5)

In [ ]:
def midpoint_betwen_to_points(p1,p2):
    p3 = tuple([(p1[i]+p2[i])/2 for i,cor in enumerate(p1)])
    return p3

p1 = APoint(float_tuple_from_str(channels_table['start'][1]))
p2 = APoint(float_tuple_from_str(channels_table['end'][1]))
p3 = APoint(midpoint_betwen_to_points(p1,p2))
print(p1)
print(p2)
print(p3)

APoint(517.08, 78.03, 10.00)
APoint(-238.99, 1373.55, 3.00)
APoint(139.05, 725.79, 6.50)


In [ ]:
acad = Autocad(create_if_not_exists=True)

def dwg_objects_sorting(acad):
    '''
    running on the dwg and sorting it's objects:
    '''
    i = 0
    j = 0
    pipes_table = pd.DataFrame(columns=['pipe #', 'start', 'end', 'pipe type',
        'nominal dia', 'id (mm)', 'length (m)', 'static head (endZ-startZ)', 
        'flow', 'consumption', 'velocity', 'head loss', 'total head loss', 'start with', 'end with'])
    pumps_table = pd.DataFrame(columns=['pump #', 'center', 'flow' , 'head','efficiency','start num','power','wetpit min volume','connect to pipe'])

    for obj in acad.iter_objects_fast():

        # print("Type of object: " + obj.ObjectName)
        
        if obj.ObjectName == "AcDbLine":
            
            # ### Add only a new pipes!!!  ### comment out becuse it's not tolerate with changes!
            # if pipes_table[(pipes_table['start'] == str(obj.Startpoint)) & (pipes_table['end'] == str(obj.Endpoint))].empty:
            
            if obj.Layer.startswith('Pipe_'):
                # if obj.Startpoint in pipes_table & obj
                pipe_name = 'Pipe '+ str(i+1)
                print(pipe_name)
                i += 1
                tmp,pipetype, nd , consumption= obj.Layer.split('_')
                length=obj.Length
                static_head=obj.Endpoint[2]-obj.Startpoint[2]
                pipe = entities.Pipe(pipetype=pipetype,nominal_dia=float(nd),length=length,static_head=static_head)
                pipe.inside_dia = pipe.inside_dia_from_nominal(nd=pipe.nominal_dia)

                data = [pipe_name,obj.Startpoint,obj.Endpoint,pipetype,nd,pipe.inside_dia,pipe.length,
                            pipe.static_head,"",consumption,"",
                            "","","",""]
                pipes_table.loc[len(pipes_table)] = data
                return pipes_table

channels_table = dwg_objects_sorting(acad=acad)
channels_table

Pipe 1


,pipe #,start,end,pipe type,nominal dia,id (mm),length (m),static head (endZ-startZ),flow,consumption,velocity,head loss,total head loss,start with,end with
0,Pipe 1,"(21.5668043743036, 11.188186431626036, -5.0)","(517.0784291680882, 78.03300443738927, 10.0)",PE100-16,800,654.8,500.224949,15.0,,500,,,,,


In [ ]:
pump_flow = [print (i) for i in channels_table['consumption']]
print(pump_flow)

500
100
300
500
[None, None, None, None]


In [ ]:

[sum(x for x in channels_table['consumption'])]
    

[1400]

In [ ]:
channels_table

,pipe #,start,end,pipe type,nominal dia,id (mm),length (m),static head (endZ-startZ),flow,consumption,velocity,head loss,total head loss,start with,end with
pipe #,,,,,,,,,,,,,,,
Pipe 1,Pipe 1,"(21.5668043743036, 11.188186431626036, -5.0)","(517.0784291680882, 78.03300443738927, 10.0)",PE100-16,800,654.8,500.224949,15.0,,500,,,,,
Pipe 2,Pipe 2,"(517.0784291680882, 78.03300443738927, 10.0)","(-238.98518323144583, 1373.551361700368, 3.0)",Steel,20,484.0,1500.016333,-7.0,,100,,,,,
Pipe 3,Pipe 3,"(-238.98518323144583, 1373.551361700368, 3.0)","(-414.49772705744687, 1070.7389660496515, 5.0)",PE100-10,500,440.6,350.005714,2.0,,300,,,,,
Pipe 4,Pipe 4,"(-414.49772705744687, 1070.7389660496515, 5.0)","(-1225.3991087248933, 1332.1290115407974, 10.0)",PE100-12.5,315,268.6,852.003995,5.0,,500,,,,,


In [ ]:
des_vel = 2
for i in range(len(channels_table)):
    pipetype = channels_table['pipe type'][i]
    flow = channels_table['flow'][i]
    length = channels_table['length (m)'][i]
    static_head = channels_table['static head (endZ-startZ)'][i]

    pipe1 = entities.Pipe(pipetype=pipetype,flow_rate=flow)
    pipe1.nominal_dia, pipe1.inside_dia, min_wt = pipe1.select_pipe_dia_from_velocity(des_vel)
    print(pipe1.pipetype, '. ID = ', pipe1.inside_dia)
    velocity = pipe1.velocity()
    head_loss = pipe1.major_head_loss()
    total_headloss = pipe1.total_head_loss()
    # print(velocity, '', head loss, '', total_headloss)

    channels_table['velocity'][i] = velocity
    channels_table['head loss'][i] = head_loss
    channels_table['total head loss'][i] = total_headloss
    channels_table['nominal dia'][i] = pipe1.nominal_dia
    channels_table['id (mm)'][i] = pipe1.nominal_dia

channels_table['total head loss']

PE100-16 . ID =  515.6
1400.0
1400.0
Steel . ID =  433
900.0
900.0
PE100-10 . ID =  396.6
800.0
800.0


C:\Users\avner\AppData\Local\Temp\ipykernel_19320\1311536993.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['velocity'][i] = velocity
C:\Users\avner\AppData\Local\Temp\ipykernel_19320\1311536993.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['head loss'][i] = head_loss
C:\Users\avner\AppData\Local\Temp\ipykernel_19320\1311536993.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['total 

PE100-12.5 . ID =  302.8
500.0
500.0


C:\Users\avner\AppData\Local\Temp\ipykernel_19320\1311536993.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['total head loss'][i] = total_headloss
C:\Users\avner\AppData\Local\Temp\ipykernel_19320\1311536993.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['nominal dia'][i] = pipe1.nominal_dia
C:\Users\avner\AppData\Local\Temp\ipykernel_19320\1311536993.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

0    0.182591
1    0.152858
2    0.172253
3    0.200991
Name: total head loss, dtype: float64

In [ ]:
# des_vel
# # pipe1.area_from_velocity(des_vel)
# pipe1.flow_rate
channels_table

,pipe #,start,end,pipe type,nominal dia,id (mm),length (m),static head (endZ-startZ),flow,consumption,velocity,head loss,total head loss,start with,end with
0,Pipe 1,"(21.5668043743036, 11.188186431626036, -5.0)","(517.0784291680882, 78.03300443738927, 10.0)",PE100-16,630,630.0,500.224949,15,1400,500,1.862558,0.005703,0.182591,Pump 1,Pipe 2
1,Pipe 2,"(517.0784291680882, 78.03300443738927, 10.0)","(-238.98518323144583, 1373.551361700368, 3.0)",Steel,18,18.0,1500.016333,-7,900,100,1.697752,0.005888,0.152858,Pipe 1,Pipe 3
2,Pipe 3,"(-238.98518323144583, 1373.551361700368, 3.0)","(-414.49772705744687, 1070.7389660496515, 5.0)",PE100-10,450,450.0,350.005714,2,800,300,1.798839,0.007261,0.172253,Pipe 2,Pipe 4
3,Pipe 4,"(-414.49772705744687, 1070.7389660496515, 5.0)","(-1225.3991087248933, 1332.1290115407974, 10.0)",PE100-12.5,355,355.0,852.003995,5,500,500,1.928705,0.011316,0.200991,Pipe 3,NaN


In [ ]:
a = channels_table[['id (mm)','total head loss']]
# pumps_table
a


,id (mm),total head loss
0,630.0,0.182591
1,18.0,0.152858
2,450.0,0.172253
3,355.0,0.200991


In [ ]:
sum(channels_table['total head loss'])

NameError: name 'pipes_table' is not defined

In [ ]:
def get_string_from_prompt(acad,prompt_str):
    a = acad.doc.Utility.GetString(1,prompt_str)
    return a

In [ ]:
# acad = win32com.client.Dispatch("AutoCAD.Application") 
# acad.Visible = True
b = autocad_functions.get_string_from_prompt(acad,'Please input something: ')

b
# a
# acad.get_selection('dsa')

'sdf'

In [ ]:
LayerObj = acad.ActiveDocument.Layers.Add('a')


In [ ]:
acad = Autocad(create_if_not_exists=True)


'''
running on the dwg and sorting it's objects:
'''
i = 0
j = 0
channels_table = pd.DataFrame(columns=['pipe #', 'start', 'end', 'pipe type',
    'nominal dia', 'id (mm)', 'length (m)', 'static head (endZ-startZ)', 
    'flow', 'consumption', 'velocity', 'head loss', 'total head loss', 'start with', 'end with'])
pumps_table = pd.DataFrame(columns=['pump #', 'center', 'flow' , 'head','efficiency','start num','power','wetpit min volume','connect to pipe'])

for obj in acad.iter_objects_fast():

    print("Type of object: " + obj.ObjectName + obj.Layer)

    if obj.Layer.startswith('Pipe'):
        # if obj.Startpoint in pipes_table & obj
        pipe_name = 'Pipe '+ str(i+1)
        i += 1
        # tmp,pipetype, nd , consumption= obj.Layer.split('_')
        length=obj.Length
        static_head=obj.Endpoint[2]-obj.Startpoint[2]
        pipetype = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter pipe type: [Steel,PE100-16,PE100-12.5,PE100-10: ')
        print(pipetype)
        nd = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter nominal diameter: ')
        print(nd)
        consumption = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter consumption: ')
        print(consumption)
        pipe = entities.Pipe(pipetype=pipetype,nominal_dia=float(nd),length=length,static_head=static_head)
        print(1)
        pipe.inside_dia = pipe.inside_dia_from_nominal(nd=pipe.nominal_dia)
        print(2)
        data = [pipe_name,obj.Startpoint,obj.Endpoint,pipetype,nd,pipe.inside_dia,pipe.length,
                    pipe.static_head,"",float(consumption),"",
                    "","","",""]
        print(3)
        channels_table.loc[len(channels_table)] = data

    # if obj.ObjectName == 'AcDbBlockReference':
    #     print(obj.Rotation*57.2957795)
    #     for obj1 in obj.iter_objects_fast():
    #         print(1)

    # i = 0
    if (obj.Layer.startswith('Pump')) & (obj.ObjectName == "AcDbCircle"):
        pump_name = 'Pump '+ str(j+1)
        j += 1
        # print(pipes_table['consumption'].sum())
        # flow = pipes_table['consumption'].sum()
        # pump = entities.Pump(rated_flow=flow)
        efficiency = autocad_functions.get_string_from_prompt(acad,'enter pump efficiency (if blank eff= 75%): ') # Will change in the futere 
        print(type(efficiency))
        if (efficiency.isdigit()):
            efficiency = float(efficiency)
        else:
            efficiency = 0.75
        print(efficiency)
        # wetpit = pump.min_wet_pit(flow,8)
        data = [pump_name,obj.Center,'',0,efficiency,'','','','']
        pumps_table.loc[len(pumps_table)] = data
        #'pump #', 'center', 'flow' , 'head','efficiency','start num','power','wetpit min volume','connect to pipe'
# pipes_table['start']

channels_table.set_index(channels_table['pipe #'],inplace=True)
pumps_table.set_index(pumps_table['pump #'],inplace=True)


Type of object: AcDbCirclePump
<class 'str'>
0.75
Type of object: AcDbLinePipe_PE100-16_800_500
Steel
20
300
1
2
3
Type of object: AcDbLinePipe_Steel_20_100


c:\Users\avner\google drive\Python\Apps\Avner autocad network solver\entities\entities.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pipe_table['Id'][0]=0


PE100-16
315
500
1
2
3
Type of object: AcDbLinePipe_PE100-10_500_300
PE100-10
225
100
1
2
3
Type of object: AcDbLinePipe_PE100-12.5_315_500
PE100-10
110
50
1
2
3


In [ ]:
pipetype = autocad_functions.get_string_from_prompt(acad,pipe_name + ' Enter pipe type or ([Steel],[PE100-16],[PE100-12.5],[PE100-10]): ')
pipetype

NameError: name 'pipe_name' is not defined

In [ ]:
def get_keyword_from_prompt(acad,prompt_str,var_list):
    acad.doc.Utility.InitializeUserInput (1, var_list)
    user_input = acad.doc.Utility.GetKeyword(prompt_str + var_list)
    return user_input

pipe_types = ['Steel', 'PE100-16', 'PE100-12.5', 'PE100-10']
# pipetype_list = 
get_keyword_from_prompt(acad,'Enter pipe type ',pipe_types)

COMError: (-2147352567, 'Exception occurred.', ('Invalid argument KeyWordList in InitializeUserInput', 'AutoCAD', 'C:\\Program Files\\Autodesk\\AutoCAD 2021\\HELP\\OLE_ERR.CHM', -2145320939, None))

In [ ]:
# pipes_type_table = pd.read_excel('data\\info\\pipes.xlsx',sheet_name=None).keys()
xl = pd.ExcelFile('data\\info\\pipes.xlsx')
pipes_type_table = pd.ExcelFile('data\\info\\pipes.xlsx').sheet_names
pipes_type_table
# xl.sheet_names  # see all sheet names

# xl.parse(sheet_name)  # read a specific sheet to DataFrame

['Steel', 'PE100-16', 'PE100-10', 'PE100-12.5']

In [ ]:
#### Lower cased pipe type:
# pipes_type_table = [x.lower() for x in pd.ExcelFile('data\\info\\pipes.xlsx').sheet_names]
# pipes_type_table

#### Sriping string:
# a = 'pe100-16'
# a.replace('pe100-','')
# # a

# pipes_type_table = pd.ExcelFile('data\\info\\pipes.xlsx').sheet_names
# # pipes_type_table_nu = [x.lower().replace('pe100-','') for x in pipes_type_table]
# # pipes_type_table_nu

# # '16' in pipes_type_table
# pipetype='PE100-16'
# pipes_type_dict = pd.read_excel('data\\info\\pipes.xlsx',sheet_name=None)

# pipes_type_dict[pipetype]['Id'].values

# a = list(pipes_type_dict['PE100-16']['ND'].values)
# a

# nominal_dia = []
# # [nominal_dia.append(pipes_type_dict[pipetype]['ND']) for x in pipes_type_dict[pipetype]['Id'] if float(pipes_type_dict[pipetype]['Id'].values) > 0]
# # nominal_dia
# list(pipes_type_dict[pipetype][pipes_type_dict[pipetype]['Id'] > 0]['ND'])

# def pipe_diameter_table (self):
#     pipe_table = pd.read_excel('pipes.xlsx',sheet_name=pipetype).fillna(value=0)
#     if pipetype == 'Steel':
#         a = list(pipe_table['ND'].drop_duplicates().values)
#         b = [str(x) for x in a]
#         return b
#     else:
#         # print(pipe_table[(pipe_table['Id']!= 0) or ()])
#         a = list(pipe_table[pipe_table['Id']!= 0]['ND'].values)
#         b = [str(x) for x in a]
#         return b
# type(a)
# 'steel' in [x.lower() for x in pipes_type_table.keys()]


# for obj in acad.get_selection("please select new pipe:"):
#     print(obj.Layer)
# pipetype = 'Steel'
# nd = '20'
# consumption = 100
# f'{pipetype}_{nd}_{consumption}'

pipes_type_dict = pd.read_excel('data\\info\\pipes.xlsx',sheet_name=None)
a

AttributeError: 'dict' object has no attribute 'fillna'

In [ ]:
acad = Autocad(create_if_not_exists=True)


pipes_type_dict = pd.read_excel('data\\info\\pipes.xlsx',sheet_name=None)
# pipes_type_table = pd.ExcelFile('data\\info\\pipes.xlsx').sheet_names
# pipes_type_table_l = [x.lower() for x in pipes_type_table]


def make_a_pipe(acad,pipe_name: str ):
    
    def get_user_pipe_type():
        
        pipetype = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter pipe type: + {pipes_type_dict.keys()}:')
        
        if (pipetype.lower() not in [x.lower() for x in pipes_type_dict.keys()]):
            
            while True:

                if pipetype.lower() == 's':
                    pipetype = 'Steel'
                    break

                if pipetype.lower() in [x.lower().replace('pe100-','') for x in pipes_type_dict.keys()]:
                    pipetype = f'PE100-{pipetype}'
                    break

                pipetype = autocad_functions.get_string_from_prompt(acad,f'WRONG INPUT!\nEnter pipe type: + {pipes_type_dict.keys()}:')
                
                if pipetype.lower() in [x.lower() for x in pipes_type_dict.keys()]:
                    break
        if pipetype == 'steel':
            pipetype.capitalize()
        return pipetype

    def pipe_diameter_table (pipetype: str) -> list:
        if pipetype == 'Steel':
            a = list(pipes_type_dict[pipetype]['ND'].drop_duplicates().values)
            b = [str(x) for x in a]
            return b
        else:
            # print(pipe_table[(pipe_table['Id']!= 0) or ()])
            a = list(pipes_type_dict[pipetype][pipes_type_dict[pipetype]['Id'] > 0]['ND'])
            b = [str(x) for x in a]
            return b

    def get_user_nominal_diameter(pipetype):
        print(1)
        pipe_diameter_table1 = pipe_diameter_table(pipetype)
        print(2)
        nominal_dia = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter Nominal Diameter: + {pipe_diameter_table1}:')
        print(3)
        while nominal_dia not in pipe_diameter_table1:

            nominal_dia = autocad_functions.get_string_from_prompt(acad,f'WRONG INPUT!\nEnter Nominal Diameter: + {pipe_diameter_table1}:')
        
        return nominal_dia

    def get_user_consumption ():
        consumption = autocad_functions.get_string_from_prompt(acad,f'Enter End Consumption in m^3/hr:')
        while True:
            try:
                float (consumption)
                break
            except:
                consumption = autocad_functions.get_string_from_prompt(acad,f'Error!!\nEnter End Consumption in m^3/hr:')
        return consumption

    def create_layer(pipetype:str, nd:int, consumption:str) -> str:
        layer_name = f"{pipetype}_{nd}_{consumption}"
        acad.ActiveDocument.Layers.Add(layer_name)
        return layer_name
        # a = acad.ActiveDocument.ChangeLayer(pipe_layer)
    
    for obj in acad.get_selection("please select new pipe:"):
        print(obj.Layer)
        pipetype = get_user_pipe_type()
        nd = get_user_nominal_diameter(pipetype)
        consumption = get_user_consumption()
        layer_name = create_layer(pipetype, nd, consumption)
        obj.Layer = layer_name
    # print(pipetype + nd + consumption)
    # nd = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter nominal diameter: ')
    # print(nd)
    # consumption = autocad_functions.get_string_from_prompt(acad,f'{pipe_name}Enter consumption: ')
    # print(consumption)

make_a_pipe(acad,'1')
# print (a)

please select new pipe:
please select new pipe:
0
1
2
3


# draw a section from pipes in autocad:


In [ ]:
p1 = acad.aDouble((0,0,0))
p2 = acad.aDouble((50,8,0))
acad.model.AddLine(p1,p2)

In [ ]:
channels_table['start'][0].replace(')',"").split(', ')[2]

'-5.0'

In [ ]:
def round_by_base(x, base):
    return base * round(x/base)

def round_to_next_i(n,i):
    if n >= 0:
        rounded_n = n + (i - n) % i
    else:
        rounded_n = n - (i + n) % i
    return rounded_n

round_to_next_i(80,100)

100

In [ ]:
-12 - (5-12) % 5

-15

In [ ]:
sum(channels_table['length (m)'])

3202.2509913977237

In [ ]:
# i = 1
# pipes_type_table = pd.read_excel('data\\info\\pipes.xlsx',sheet_name='Steel')
# # od = pipes_type_table[pipes_type_table["ND"] == pipes_table['nominal dia'][i]]['OD'].iloc[0]/1000 #float only in jupeter (excel)
# # pipe_invert_start = float_tuple_from_str(pipes_table['start'][i])[2] # in app dont' need the float and cleaning part. only for jupyter
# # pipe_invert_end = float_tuple_from_str(pipes_table['end'][i])[2]
# # od
# # pipes_table['nominal dia'][i]
# min_wt = pipes_type_table[pipes_type_table['ND']==pipes_table['nominal dia'][i]]['wall thickness'].min()/1000
# min_wt

0.0039700000000000004

In [ ]:

####################### Def - draw_pipe_sec (acad, pipes_table)


def draw_pipe_sec (acad, pipes_table):
    total_length = 0 # sum of all pipe_length to point

    for i in range(len(pipes_table)):
        
        ### data ###
        pipe_invert_start = float_tuple_from_str(pipes_table['start'][i])[2] # in app dont' need the float and cleaning part. only for jupyter
        pipe_invert_end = float_tuple_from_str(pipes_table['end'][i])[2] #float(pipes_table['end'][i].replace(')',"").split(', ')[2])
        pipe_length = pipes_table['length (m)'][i]
        pipe_type = pipes_table['pipe type'][i]

        if pipe_type == 'Steel':
            pipes_type_table = pd.read_excel('data\\info\\pipes.xlsx',sheet_name='Steel')
            od = pipes_type_table[pipes_type_table["ND"] == pipes_table['nominal dia'][i]]['OD'].iloc[0]/1000 #float only in jupeter (excel)
            min_wt = pipes_type_table[pipes_type_table['ND']==pipes_table['nominal dia'][i]]['wall thickness'].min()/1000
        else:
            od = pipes_table['nominal dia'][i]/1000
            min_wt = ((pipes_table['nominal dia'][i] - pipes_table['id (mm)'][i])/2)/1000
            
        ### invert line ###   
        p1 = APoint(total_length,pipe_invert_start)
        p2 = APoint((total_length + pipe_length),pipe_invert_end)
        acad.model.AddLine(p1,p2)

        ### down out side line
        p3 = APoint(total_length,pipe_invert_start - min_wt)
        p4 = APoint((total_length + pipe_length),pipe_invert_end - min_wt)
        acad.model.AddLine(p3,p4) 
        
        ### up out side line
        p5 = APoint(total_length,pipe_invert_start - min_wt + od)
        p6 = APoint((total_length + pipe_length),pipe_invert_end - min_wt + od)
        acad.model.AddLine(p5,p6) 

        ### up inside side line
        p7 = APoint(total_length,pipe_invert_start - min_wt*2 + od)
        p8 = APoint((total_length + pipe_length),pipe_invert_end - min_wt*2 + od)
        acad.model.AddLine(p7,p8) 

        ####### Reset to next pipe #########
        total_length += pipe_length       




In [ ]:
pipes_type_table = pd.read_excel('data\\info\\pipes.xlsx',sheet_name='Steel')
od = pipes_type_table[pipes_type_table["ND"] == channels_table['nominal dia'][1]]['OD'].iloc[0]
od
# pipes_table['nominal dia']

406.4

In [ ]:
px1 = APoint((0,1,0))
px2 = APoint((0,22,0))
px3 = APoint(px1.x,px1.y-5)
px3
text = acad.model.Addtext(f'1000 m',px3,2.5)
# text.Alignment.AlignmentLeft
text.Alignment=4


In [ ]:
def round_by_base(x, base):
    return base * round(x/base)

def round_to_next_i(n,i):
    if n >= 0:
        rounded_n = n + (i - n) % i
    else:
        rounded_n = n - (i + n) % i
    return rounded_n

##############################################

####################### Def - make_a_sec_grid (acad, pipes_table, x_steps, y_steps)
def make_a_sec_grid1 (acad, pipes_table, x_steps, y_steps):
    max_y = round_to_next_i(pipes_table['static head (endZ-startZ)'].max(),y_steps)+y_steps
    min_y = round_to_next_i(pipes_table['static head (endZ-startZ)'].min(),y_steps)-y_steps

    min_x = 0
    max_x = round_to_next_i(int(sum(pipes_table['length (m)'])),x_steps)

    py1 = APoint((0,min_y,0))
    py2 = APoint((max_x,min_y,0))
    py3 = APoint((-5,min_y,0))

    px1 = APoint((0,min_y,0))
    px2 = APoint((0,min_y+y_steps/2,0))
    px3 = APoint(px1.x,px1.y-5,0)

    for y in range (min_y,max_y+y_steps,y_steps):

        acad.model.AddLine(py1,py2)
        text = acad.model.Addtext(py1.y,py1,2.5)
        
        py1.y += y_steps
        py2.y += y_steps
        py3.y += y_steps

    for x in range (min_x, max_x+1,x_steps):    

        acad.model.AddLine(px1,px2)
        # text.Alignment = 1
        text = acad.model.Addtext(f'{x} m',px3,2.5)
        

        px1.x += x_steps
        px2.x += x_steps
        px3.x += x_steps

x_steps = 500
y_steps = 5
make_a_sec_grid1(acad,channels_table,x_steps,y_steps)
# draw_pipe_sec(acad,pipes_table)

In [ ]:
y_steps = 5
max_y = round_to_next_i(channels_table['static head (endZ-startZ)'].max(),y_steps)
max_y

15

In [ ]:
def round_by_base(x, base):
    return base * round(x/base)

def round_to_next_i(n,i):
    if n >= 0:
        rounded_n = n + (i - n) % i
    else:
        rounded_n = n - (i + n) % i
    return rounded_n

##############################################
####################### Autocad-layers

grid_layer = acad.ActiveDocument.Layers.Add("grid")
grid_text_layer = acad.ActiveDocument.Layers.Add("grid_text")
pipe_layer = acad.ActiveDocument.Layers.Add("pipe")
pipe_guideline_layer = acad.ActiveDocument.Layers.Add("pipe_guideline")

grid_layer.color = 7
grid_text_layer.color = 7
pipe_layer.color = 5
pipe_guideline_layer = 8

####################### Def - make_a_sec_grid (acad, pipes_table, x_steps, y_steps)
def make_a_sec_grid (acad, pipes_table, x_steps, y_steps):
    max_y = round_to_next_i(pipes_table['static head (endZ-startZ)'].max(),y_steps)+y_steps
    min_y = round_to_next_i(pipes_table['static head (endZ-startZ)'].min(),y_steps)-y_steps

    min_x = 0
    max_x = round_to_next_i(int(sum(pipes_table['length (m)'])),x_steps)

    py1 = APoint((0,min_y,0))
    py2 = APoint((max_x,min_y,0))
    py3 = APoint((-50,min_y,0))

    px1 = APoint((0,min_y,0))
    px2 = APoint((0,min_y+y_steps/2,0))
    px3 = APoint((0,max_y,0))
    px4 = APoint((px1.x-5,px1.y-5,0))
    

    for y in range (min_y,max_y+y_steps,y_steps):

        acad.ActiveDocument.ActiveLayer = grid_layer
        main_horizntel_line = acad.model.AddLine(py1,py2)
        
        acad.ActiveDocument.ActiveLayer = grid_text_layer
        text = acad.model.Addtext(py1.y,py1,2.5)
        # text.Justify = 1 
                
        py1.y += y_steps
        py2.y += y_steps
        py3.y += y_steps

    for x in range (min_x, max_x+1,x_steps):    

        acad.ActiveDocument.ActiveLayer = grid_layer
        tips_line = acad.model.AddLine(px1,px2)
        grid_vertical_line = acad.model.AddLine(px1,px3)
        grid_vertical_line.color = 8

        acad.ActiveDocument.ActiveLayer = grid_text_layer
        text = acad.model.Addtext(f'{x} m',px4,2.5)
        # text.Alignment = 0        

        px1.x += x_steps
        px2.x += x_steps
        px3.x += x_steps
        px4.x += x_steps
    
    return min_x, max_x, min_y, max_y


####################### Def - draw_pipe_sec (acad, pipes_table)
def draw_pipe_sec (acad, pipes_table,min_y,max_y):
    total_length = 0 # sum of all pipe_length to point

    for i in range(len(pipes_table)):
        
        ### data ###
        pipe_invert_start = float_tuple_from_str(pipes_table['start'][i])[2] # in app dont' need the float and cleaning part. only for jupyter
        pipe_invert_end = float_tuple_from_str(pipes_table['end'][i])[2] #float(pipes_table['end'][i].replace(')',"").split(', ')[2])
        pipe_length = pipes_table['length (m)'][i]
        pipe_type = pipes_table['pipe type'][i]

        if pipe_type == 'Steel':
            pipes_type_table = pd.read_excel('data\\info\\pipes.xlsx',sheet_name='Steel')
            od = pipes_type_table[pipes_type_table["ND"] == pipes_table['nominal dia'][i]]['OD'].iloc[0]/1000 #float only in jupeter (excel)
            min_wt = pipes_type_table[pipes_type_table['ND']==pipes_table['nominal dia'][i]]['wall thickness'].min()/1000
        else:
            od = pipes_table['nominal dia'][i]/1000
            min_wt = ((pipes_table['nominal dia'][i] - pipes_table['id (mm)'][i])/2)/1000

        ############ Draw pipe

        acad.ActiveDocument.ActiveLayer = pipe_layer
        ### invert line ###   
        p1 = APoint(total_length,pipe_invert_start)
        p2 = APoint((total_length + pipe_length),pipe_invert_end)
        acad.model.AddLine(p1,p2)

        ### down out side line
        p3 = APoint(total_length,pipe_invert_start - min_wt)
        p4 = APoint((total_length + pipe_length),pipe_invert_end - min_wt)
        acad.model.AddLine(p3,p4) 
        
        ### up out side line
        p5 = APoint(total_length,pipe_invert_start - min_wt + od)
        p6 = APoint((total_length + pipe_length),pipe_invert_end - min_wt + od)
        acad.model.AddLine(p5,p6) 

        ### up inside side line
        p7 = APoint(total_length,pipe_invert_start - min_wt*2 + od)
        p8 = APoint((total_length + pipe_length),pipe_invert_end - min_wt*2 + od)
        acad.model.AddLine(p7,p8) 

        ### pipe guideline ###
        p9 = APoint(total_length,min_y)
        p10 = APoint(total_length,max_y)
        acad.model.AddLine(p9,p10)

        ####### Reset to next pipe #########
        total_length += pipe_length       

x_steps = 500
y_steps = 5
min_x,max_x,min_y,max_y = make_a_sec_grid(acad,channels_table,x_steps,y_steps)
draw_pipe_sec(acad,channels_table,min_y,max_y)


In [ ]:

LayerObj = acad.ActiveDocument.Layers.Add("Hit_layer")
     # Add a new layer, the layer name is "HIT_Layer". 
acad.ActiveDocument.ActiveLayer = LayerObj

     # Set the "HIT_Layer" layer as the current layer.
ClrNum = 7
LayerObj.color = ClrNum
     # ClrNum is the color index number, and its value range is [0,256]; 
     # The color index number of the standard color is specified as follows: 1 red, 2 yellow, 3 green, 4 cyan, 5 blue, 6 Magenta, 7 White/Black; 
     # 0 ByBlock, 256 ByLayer; 
     # For other color index numbers, see https://wenku.baidu.com/view/9d458b70195f312b3069a505.html.



In [ ]:
grid_layer = acad.ActiveDocument.Layers.Add("grid")
grid_text_layer = acad.ActiveDocument.Layers.Add("grid_text")
pipe_layer = acad.ActiveDocument.Layers.Add("pipe")

grid_layer.color = 7
grid_text_layer.color = 7
pipe_layer.color = 5

     # Add a new layer, the layer name is "HIT_Layer". 
# acad.ActiveDocument.ActiveLayer = LayerObj

#      # Set the "HIT_Layer" layer as the current layer.
# ClrNum = 7

     # ClrNum is the color index number, and its value range is [0,256]; 
     # The color index number of the standard color is specified as follows: 1 red, 2 yellow, 3 green, 4 cyan, 5 blue, 6 Magenta, 7 White/Black; 
     # 0 ByBlock, 256 ByLayer; 
     # For other color index numbers, see https://wenku.baidu.com/view/9d458b70195f312b3069a505.html.



In [ ]:
import win32com.client as win32
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open('data\\outputs\\acad-pipelines.xlsx')
# Alternately, specify the full path to the workbook
# wb = excel.Workbooks.Open(r'C:\myfiles\excel\workbook2.xlsx')
excel.Visible = True


com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Excel', 'פעולת השירות Open ממחלקה Workbooks נכשלה', 'xlmain11.chm', 0, -2146827284), None)

In [ ]:
# def make_a_pipe_from_app(pipetype:str, nd:int, consumption:str):

#     def create_layer(pipetype:str, nd:str, consumption:str) -> str:
#         layer_name = f"Pipe_{pipetype}_{nd}_{consumption}"
#         acad.ActiveDocument.Layers.Add(layer_name)
#         return layer_name

#     for obj in acad.get_selection("please select new pipe:"):
#         # print(obj.Layer)
#         layer_name = create_layer(pipetype, nd, consumption)
#         obj.Layer = layer_name


# 


# make_a_pipe_from_app('Steel',10,300)

In [ ]:
# acad.prompt('Selcet Pipe start point:')
# start = APoint(acad.doc.Utility.GetPoint())
# acad.prompt('Selcet Pipe end point:')
# end = APoint(acad.doc.Utility.GetPoint())
# # print(start, '  ' , end)

acad.model.AddLine(APoint(acad.doc.Utility.GetPoint()),APoint(acad.doc.Utility.GetPoint()))

In [ ]:
def draw_a_pipe_from_app(pipetype:str, nd:int, consumption:str):

    def create_layer(pipetype:str, nd:str, consumption:str) -> str:
        layer_name = f"Pipe_{pipetype}_{nd}_{consumption}"
        acad.ActiveDocument.Layers.Add(layer_name)
        return layer_name

    layer_name = create_layer(pipetype, nd, consumption)
    acad.prompt('Selcet Pipe start point:')
    start = APoint(acad.doc.Utility.GetPoint())
    acad.prompt('Selcet Pipe end point:')
    end = APoint(acad.doc.Utility.GetPoint())
    new_pipe = acad.model.AddLine(start,end)
    new_pipe.Layer = layer_name

draw_a_pipe_from_app('Steel', 12, 300)

Selcet Pipe start point:
Selcet Pipe end point:


# Channels 

## get info from dwg

In [9]:
def manning_eq_flow_from_water_level(water_depth,bottom_width:float,bank_slope:float,channel_slope:float,manning_coefficient:float):
    
    h = water_depth
    b = bottom_width
    m = bank_slope
    j = channel_slope
    n = manning_coefficient

    cross_sectional_area = eq.manning_cross_sectional_area_eq(b=b,h=h,m=m)
    wetted_perimeter = eq.manning_wetted_perimeter_eq (b,h,m)
    flow_from_depth = (cross_sectional_area**(5/3)*math.sqrt(j))/(n*(wetted_perimeter)**(2/3))
    velocity = flow_from_depth/cross_sectional_area


    return flow_from_depth, velocity, wetted_perimeter,cross_sectional_area

channels_table = pd.DataFrame(columns=['channel #', 'start', 'end', 'length (m)', 'static head (endZ-startZ)', 'slope',
                                        'manning coefficent', 'flow' ,'velocity', 'water depth', 'max flow rates', 'max water depth', 'geomatry: bottom width','geometry: bank slope', 'geometry: channel depth', 'geometry: channel width', 'geometry: free board', 'start with', 'end with'])
k=0 

for obj in acad.iter_objects_fast():

    # print("Type of object: " + obj.ObjectName)
    
    if obj.ObjectName == "AcDbLine":
        
        if obj.Layer.startswith('Channel_'):
            '''
            Layer template:
            m = bank slope
            b = bottom width
            n = manning coefficient
            q = flow rate
            mwd = Max Water Depth allowed
            cd = desgined Channel Depth
            fb = Free board

            '''
            
            
            channel_name = 'Channel '+ str(k+1)
            k += 1
            
            try:
                bank_slope = float(re.search(r'(?:[_])m-(\d+)',obj.Layer).group(1))
            except:
                #Create default values - rectangle channel
                bank_slope = 0
            try:
                bottom_width = float(re.search(r'(?:[_])b-(\d*\.*\d+)',obj.Layer).group(1))
            except:
                #Create default values - Triangle channel
                bottom_width = 0
            try:
                max_water_depth = float(re.search(r'(?:[_])mwd-(\d*\.*\d+)',obj.Layer).group(1))
            except:
                max_water_depth = False
            try:
                flow_rate = float(re.search(r'(?:[_])q-(\d*\.*\d+)',obj.Layer).group(1))
            except:
                flow_rate = 0
            try:
                manning_coefficient = float(re.search(r'(?:[_])n-(\d*\.*\d+)',obj.Layer).group(1))
            except:
                #Create default values
                manning_coefficient = 0.035
            try:
                channel_depth = float(re.search(r'(?:[_])cd-(\d*\.*\d+)',obj.Layer).group(1))
                # input(channel_depth)
            except:
                #Create default values
                channel_depth = ""
            try:
                free_board = float(re.search(r'(?:[_])fb-(\d*\.*\d+)',obj.Layer).group(1))
            except:
                #Create default values
                free_board = 0.25
            print(channel_depth)
            length=float(obj.Length)
            # print(length)

            static_head=float(obj.Startpoint[2]-obj.Endpoint[2])
            # print(static_head)
            channel_slope = static_head/length
            # print(bank_slope, bottom_width, max_water_depth, flow_rate, manning_coefficient, channel_slope )
            # print(obj.Layer)
            
            channel = entities.Channel(manning_coefficient=manning_coefficient,flow_rate=flow_rate,bank_slope=bank_slope,bottom_width=bottom_width,max_water_depth=max_water_depth,channel_slope=channel_slope)

            # print('max water: ' ,max_water_depth)
            # if flow_rate > 0:
            #     # print(channel.water_depth)
            
            if channel_depth != "":
                max_water_depth = channel_depth - free_board
            
            if max_water_depth:    
                max_flow_rate,max_velocity, max_wetted_perimeter, max_cross_sectional_area = manning_eq_flow_from_water_level(max_water_depth,bottom_width,bank_slope,channel_slope,manning_coefficient)
                print(f'the max flow rate is: {max_flow_rate*3600} m^3/hr, but the desgin is for {flow_rate} m^3/hr')


            
                # if (max_water_depth < channel.water_depth):
                    # print('Channel Geometry not good for that water flow')  
            else:
                max_flow_rate=''
                max_velocity=''
                max_wetted_perimeter=''
                max_cross_sectional_area=''

            data = [channel_name,obj.Startpoint,obj.Endpoint,length, static_head,channel.channel_slope ,
                                            manning_coefficient ,channel.flow_rate,channel.velocity,channel.water_depth,
                                            max_flow_rate*3600,max_water_depth,bottom_width,channel.bank_slope,
                                            channel_depth, '',free_board, '','']
            channels_table.loc[len(channels_table)] = data


channels_table



the max flow rate is: 647.4865139418966 m^3/hr, but the desgin is for 1200.0 m^3/hr
1.2
the max flow rate is: 10101.841288290221 m^3/hr, but the desgin is for 1200.0 m^3/hr


,channel #,start,end,length (m),static head (endZ-startZ),slope,manning coefficent,flow,velocity,water depth,max flow rates,max water depth,geomatry: bottom width,geometry: bank slope,geometry: channel depth,geometry: channel width,geometry: free board,start with,end with
0,Channel 1,"(-271.0696617158046, 4062.033799129046, 10.0)","(-2747.5195659261644, 3116.571339497834, 0.0)",2650.811836,10.0,0.003772,0.035,1200.0,0.590940,0.297546,,False,1.0,3.0,,,0.25,,
1,Channel 2,"(-2747.5195659261644, 3116.571339497834, 0.0)","(-1565.336305236331, 2430.777475097332, -10.0)",1366.737168,10.0,0.007317,0.035,1200.0,0.777675,0.378418,647.486514,0.3,0.0,3.0,,,0.25,,
2,Channel 3,"(-1565.336305236331, 2430.777475097332, -10.0)","(-2357.368366961731, 1234.5574759611845, -20.0)",1434.697555,10.0,0.006970,0.035,1200.0,0.759403,0.218964,10101.841288,0.95,2.0,0.0,1.2,,0.25,,


In [116]:
txt = 'Channel_m-0_b-2_avner-1.2_q-1200_n-0.035'
# bottom_width = float(re.search(r'(?:[_])b-(\d\*.\*d+)',txt).group(1))
# channel_depth = re.search(r'(?:[_])n-(\d+\*.\*d+)',txt).group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [99]:
# def manning_eq_flow_from_water_level(water_depth,bottom_width:float,bank_slope:float,channel_slope:float,manning_coefficient:float):
    
#     h = water_depth
#     b = bottom_width
#     m = bank_slope
#     j = channel_slope
#     n = manning_coefficient

#     cross_sectional_area = eq.manning_cross_sectional_area_eq(b=b,h=h,m=m)
#     wetted_perimeter = eq.manning_wetted_perimeter_eq (b,h,m)
#     flow_from_depth = (cross_sectional_area**(5/3)*math.sqrt(j))/(n*(wetted_perimeter)**(2/3))
#     velocity = flow_from_depth/cross_sectional_area


#     return flow_from_depth, velocity, wetted_perimeter,cross_sectional_area
# print(max_water_depth,bottom_width,bank_slope, channel_slope,manning_coefficient)
# max_flow_rate,max_velocity, max_wetted_perimeter, max_cross_sectional_area = eq.manning_eq_flow_from_water_level(max_water_depth,bottom_width=bottom_width,bank_slope=bank_slope, channel_slope=channel_slope,manning_coefficient=manning_coefficient)
# max_flow_rate

In [100]:
# channels_table = pd.DataFrame(columns=['channel #', 'start', 'end', 'length (m)', 'static head (endZ-startZ)', 'slope',
#                                         'flow' ,'velocity', 'water depth', 'max flow rates', 'max water depth', 'geomatry: bottom width','geomatry: bank slope', 'geometry: channel depth', 'geometry: channel width', 'start with', 'end with'])
# data = [channel_name,obj.Startpoint,obj.Endpoint,length, static_head,channel.channel_slope ,
#                                         channel.flow_rate,channel.velocity,channel.water_depth,"",max_water_depth,bottom_width,channel.bank_slope, '', '', '','']
# data

In [101]:

# for obj in acad.iter_objects_fast():

#     # print("Type of object: " + obj.ObjectName)
    
#     if obj.ObjectName == "AcDbLine":
        
#         if obj.Layer.startswith('Channel_'):
#             print(obj.Layer)
#             max_water_depth = float(re.search(r'(?:[_])mwh-(\d*\.*\d+)',obj.Layer).group(1))
#             print(max_water_depth)

## write text to dwg

In [102]:
channel_text_layer = acad.ActiveDocument.Layers.Add("channel_text")

### Channels writing info:
acad.ActiveDocument.ActiveLayer = channel_text_layer
for i in range(len(channels_table)):

    p1 = APoint(channels_table['start'][i])

    vertex_text1 = f'Vertex number: {i+1}'
    vertex_text2 = f'Z={p1.z}'

    text = acad.model.Addtext(vertex_text1,p1,2.5)
    p1.y -= 4
    text = acad.model.Addtext(vertex_text2,p1,2.5)

    if i == (len(channels_table)-1):
        
        p4 = APoint(channels_table['end'][i])
        vertex_text1 = f'Vertex number: {i+2}'
        vertex_text2 = f'Z={p4.z}'
        
        text = acad.model.Addtext(vertex_text1,p4,2.5)
        p4.y -= 4
        text = acad.model.Addtext(vertex_text2,p4,2.5)

    p2 = APoint(channels_table['end'][i])
    p3 = APoint(uf.midpoint_betwen_to_points(p2,p1))
    
    p3.y += 80

    channel_name = str(channels_table['channel #'][i])
    text = acad.model.Addtext(channel_name,p3,2.5)
    
    p3.y -= 4
    length = f"length: {round(channels_table['length (m)'][i],2)} m"
    text = acad.model.Addtext(length,p3,2.5)

    p3.y -= 4
    slope = f"slope: {round(float(channels_table['slope'][i]*100),2)} %"
    text = acad.model.Addtext(slope,p3,2.5)

    p3.y -= 4

    des_flow = f"the water level will be: {round(float(channels_table['water depth'][i]*100),2)} cm @ design flow rate {round(float(channels_table['flow'][i]),2)}"
    text = acad.model.Addtext(des_flow,p3,2.5)

    p3.y -= 4

    if channels_table['max water depth'][i]:
        max_flow = f"the max allowed water level is: {round(float(channels_table['max water depth'][i]*100),2)} cm @ flow rate {round(float(channels_table['max flow rates'][i]),2)}"
        text = acad.model.Addtext(max_flow,p3,2.5)

        p3.y -= 4

## Draw a Channel section:

In [13]:
channel_layer = acad.ActiveDocument.Layers.Add("channel")
channel_water_layer = acad.ActiveDocument.Layers.Add("channel water")
channel_max_water_layer = acad.ActiveDocument.Layers.Add("channel max water")
channel_layer.color = 38
channel_water_layer.color = 5
channel_max_water_layer.color = 4

# print (len(channels_table))
for i in range(len(channels_table)):
        
        ### data ###
        bottom_width = channels_table['geomatry: bottom width'][i]*100
        bank_slope = channels_table['geometry: bank slope'][i]
        water_depth = channels_table['water depth'][i]*100
        flow_rate = channels_table['flow'][i]*100
        free_board =  channels_table['geometry: free board'][i]*100
        channel_depth = channels_table['geometry: channel depth'][i]*100
        max_water_depth = channels_table['max water depth'][i]*100
        max_flow_rate = channels_table['max flow rates'][i]*100

        try:
                channel_depth = float(channel_depth)
                # print(1)
        except:
                try:
                        if max_water_depth:
                                channel_depth = float (max_water_depth)+float(free_board)

                        else:
                                channel_depth = float(channel_depth)
                        # print(channel_depth)
                except:
                        channel_depth = float (water_depth)+free_board
                        # print(3)
        
        # print(channel_depth)
        # print (water_depth)
        # print (max_water_depth)
        side_steps = (channel_depth/100)*float(bank_slope)*100 
        water_side_steps = (water_depth/100)*float(bank_slope)*100
        max_water_side_steps = ((max_water_depth/100)*float(bank_slope))*100
        # print(side_steps , water_side_steps, max_water_side_steps)
        
        ############ Draw sec

        acad.ActiveDocument.ActiveLayer = channel_layer
        
        ps = APoint(channels_table['start'][i])
        pe = APoint(channels_table['end'][i])
        p3 = APoint(uf.midpoint_betwen_to_points(pe,ps))

        p3.y += 100 #Set (0,0) of section
        p3.z = 0

        p2 = APoint((p3.x - side_steps),(p3.y + channel_depth))
        p1 = APoint((p2.x-100), p2.y)
        # print (p1)
        # print (p2)
        # print (p3)
        if bottom_width != 0:
                p4 = APoint((p3.x + bottom_width),p3.y)
                p5 = APoint((p4.x + side_steps),(p4.y + channel_depth))
                p6 = APoint((p5.x+100),p5.y)
                p7 = APoint((p4.x + water_side_steps), (p4.y + water_depth))
                p8 = APoint((p4.x + max_water_side_steps), (p4.y + max_water_depth))
        else:
                p4 = APoint((p3.x + side_steps),(p3.y + channel_depth))
                p5 = APoint((p4.x+100),p4.y)
                p6 = p5
                p7 = APoint((p3.x + water_side_steps), (p3.y + water_depth))
                p8 = APoint((p3.x + max_water_side_steps), (p3.y + max_water_depth))


        acad.model.AddLine(p1,p2)
        acad.model.AddLine(p2,p3)
        acad.model.AddLine(p3,p4)
        acad.model.AddLine(p4,p5)
        acad.model.AddLine(p5,p6)

        ############ Draw water levels

        acad.ActiveDocument.ActiveLayer = channel_water_layer
        
        p9 = APoint((p3.x - water_side_steps),(p3.y + water_depth))
        water_line = acad.model.AddLine(p7,p9)
        water_text_point = APoint(uf.midpoint_betwen_to_points(p7,p9))
        water_text_point.x -= 50
        des_flow = f"the water level will be: {round(water_depth,2)} cm @ design flow rate {round(flow_rate,2)}"
        text = acad.model.Addtext(des_flow,water_text_point,2.5)

        if max_water_depth:
                acad.ActiveDocument.ActiveLayer = channel_max_water_layer
                p10 = APoint((p3.x - max_water_side_steps),(p3.y + max_water_depth))
                max_water_line = acad.model.AddLine(p8,p10)
                max_water_text_point = APoint(uf.midpoint_betwen_to_points(p8,p10))
                max_water_text_point.x -= 50
                max_flow = f"the water level will be: {round(max_water_depth,2)} cm @ design flow rate {round(max_flow_rate,2)}"
                text = acad.model.Addtext(des_flow,max_water_text_point,2.5)

                if (max_water_depth < water_depth):
                        
                        max_water_text_point.y -= 10
                        max_water_text_point.x -= 50
                        bad_geometry_text ='Channel Geometry is not good for the designed water flow'
                        text1 = acad.model.Addtext(bad_geometry_text,max_water_text_point,5)
                        text1.color = 1




In [70]:
l1 = acad.model.AddLine(p8,p10)
l1.StartPoint

(-2066.4279355812478, 2903.674407297583, 0.0)

In [51]:

# for i in range(len(channels_table)):
        
#         ### data ###
#         bottom_width = channels_table['geomatry: bottom width'][i]*100
#         bank_slope = channels_table['geometry: bank slope'][i]
#         water_depth =  channels_table['water depth'][i]*100
#         free_board =  channels_table['geometry: free board'][i]*100
#         channel_depth = channels_table['geometry: channel depth'][i]*100
#         max_water_depth = channels_table['max water depth'][i]*100
#         print(f'''bottom_width= {bottom_width}\n 
#                     bank_slope={bank_slope}
#                     water_depth={water_depth} 
#                     free_board = {free_board}
#                     channel_depth = {channel_depth}
#                     max_water_depth = {max_water_depth}''')

NameError: name 'p5' is not defined

In [80]:
from utils.autocad import pipes_network_sytems
from utils.autocad import autocad_analyzing
from entities import entities
import pandas as pd
import os

pipes_table = pd.DataFrame()
pumps_table = pd.DataFrame()
channels_table = pd.DataFrame()

pipes_table, pumps_table, channels_table = autocad_analyzing.dwg_objects_sorting()
autocad_analyzing.is_pipe_conected(pipes_table,pumps_table)

with pd.ExcelWriter('data\\outputs\\acad-pipelines.xlsx') as writer:
        pipes_table.to_excel(writer,sheet_name='Pipes',index=False)
        pumps_table.to_excel(writer,sheet_name='Pumps',index=False)
        channels_table.to_excel(writer,sheet_name='Channels',index=False)    
os.startfile('data\\outputs\\acad-pipelines.xlsx')    

c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\entities\entities.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pipe_table['Id'][0]=0
c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\entities\entities.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.pipe_table['Id'][0]=0
c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\entities\entities.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

the max flow rate is: 647.4865139418966 m^3/hr, but the desgin is for 1200.0 m^3/hr
Channel depth 1.2
the max flow rate is: 10101.841288290221 m^3/hr, but the desgin is for 1200.0 m^3/hr


c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\utils\autocad\autocad_analyzing.py:236: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['start with'].iloc[i] = start_with_pump_name
c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\utils\autocad\autocad_analyzing.py:237: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pumps_table['connect to pipe'][start_with_pump_name] = pipes_table['pipe #'].iloc[i]
c:\Users\Avner.PALGEY-MAIM\python-avner\Avner autocad network solver\utils\autocad\autocad_analyzing.py:243: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [49]:
pipes_table

,pipe #,start,end,pipe type,nominal dia,id (mm),length (m),static head (endZ-startZ),flow,consumption,minimum pressure required,velocity,head loss,total head loss,Pressure at end of pipe,start with,end with,costs per meter,total costs
pipe #,,,,,,,,,,,,,,,,,,,
Pipe 1,Pipe 1,"(536.08593755699, 1927.8727861776483, -5.0)","(1031.5975623507745, 1994.7176041834116, 10.0)",PE100-16,800.0,654.8,500.224949,15.000000,,500.0,0.0,,,,,Pump 1,Pipe 2,3684.020571,1.842839e+06
Pipe 2,Pipe 2,"(1031.5975623507745, 1994.7176041834116, 10.0)","(275.5339499512405, 3290.2359614463903, 3.0)",Steel,20.0,484.0,1500.016333,-7.000000,,0.0,0.0,,,,,Pipe 1,Pipe 3,1140.000000,1.710019e+06
Pipe 3,Pipe 3,"(275.5339499512405, 3290.2359614463903, 3.0)","(100.02140612523948, 2987.4235657956738, 5.0)",PE100-10,500.0,440.6,350.005714,2.000000,,300.0,70.0,,,,,Pipe 2,"Pipe 4, Pipe 6",1200.000000,4.200069e+05
Pipe 4,Pipe 4,"(100.02140612523948, 2987.4235657956738, 5.0)","(-710.8799755422069, 3248.8136112868197, 10.0)",PE100-12.5,315.0,268.6,852.003995,5.000000,,500.0,20.0,,,,,Pipe 3,Pipe 5,640.000000,5.452826e+05
Pipe 5,Pipe 5,"(-710.8799755422069, 3248.8136112868197, 10.0)","(-963.7318520066838, 2783.554092127968, 0.0)",Steel,10.0,253.0,529.622971,-10.000000,,300.0,0.0,,,,,Pipe 4,,520.000000,2.754039e+05
Pipe 6,Pipe 6,"(100.02140612523948, 2987.4235657956738, 5.0)","(-224.57043511596868, 2427.404145460851, 8.698...",Steel,10.0,253.0,647.298459,3.698788,,300.0,0.0,,,,,Pipe 3,Pipe 7,520.000000,3.365952e+05
Pipe 7,Pipe 7,"(-224.57043511596868, 2427.404145460851, 8.698...","(-749.0632077821901, 2530.194204976131, 10.0)",PE100-12.5,315.0,268.6,534.471850,1.301212,,500.0,20.0,,,,,Pipe 6,,640.000000,3.420620e+05


In [50]:
# pipes_table[len(pipes_table['end with'].str.split(','))>1]
pipes_table['end with']
pipes_table[pipes_table['end with'].str.contains(',', na=False)]['pipe #']
# print(filtered_df)

pipe #
Pipe 3    Pipe 3
Name: pipe #, dtype: object

In [81]:
junctions_list= list(pipes_table[pipes_table['end with'].str.contains(',', na=False)]['pipe #'])
junctions_list

# new_pipe = 'Pipe 7'
# pipes_table['end with'][new_pipe]

['Pipe 3']

In [52]:
# # only split, not ring

# branches = {}
# for junction in junctions_list:
#     branched_pipes = pipes_table['end with'][junction].split(', ')
#     branches[junction] = {}
#     for pipes in branched_pipes:
#         print(pipes)
#         for i,pipe in enumerate(pipes):
#             print(pipe)
#             branch_id = i
#             new_pipe=pipe
#             branch_pipes = ''
#             branch_consumption = 0
#             while pipes_table['end with'][new_pipe] != '':
#                 branch_consumption += pipes_table['consumption'][new_pipe]
#                 branch_pipes += str(new_pipe)
#                 new_pipe= pipes_table['end with'][new_pipe]
#             branch_consumption += pipes_table['consumption'][new_pipe]
#             branch_pipes += str(new_pipe)
#             print(branch_consumption)
#             branch_dict = {
#                 'pipes' : branch_pipes,
#                 'branch_consumption' : branch_consumption
#             }

#             branches[junction][branch_id] = branch_dict
#             print(branches)
#         pass
# branches

In [84]:
# only split, not ring

branches_tree = {}
for junction in junctions_list:
    branched_pipes = pipes_table['end with'][junction].split(', ')
    branches_tree[junction] = {}
    for i,pipe in enumerate(branched_pipes):
        # print(pipe)
        # for i,pipe in enumerate(pipes):
        #     print(pipe)
        branch_id = f'Branch {i+1}'
        new_pipe=pipe
        branch_pipes = []
        branch_consumption = 0
        while pipes_table['end with'][new_pipe] != '':
            branch_consumption += pipes_table['consumption'][new_pipe]
            branch_pipes.append(str(new_pipe))
            new_pipe= pipes_table['end with'][new_pipe]
        branch_consumption += pipes_table['consumption'][new_pipe]
        branch_pipes.append(str(new_pipe))
        print(branch_consumption)
      
        pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption
        branch_dict = {
            'pipes' : branch_pipes,
            'branch_consumption' : branch_consumption,
            'tested': False,
        }

        branches_tree[junction][branch_id] = branch_dict
        # print(branches)
        # pass
branches_tree
# pipes_table

800.0
800.0


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\498842354.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\498842354.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][[branch_pipes][0][0]]= branch_consumption


{'Pipe 3': {'Branch 1': {'pipes': ['Pipe 4', 'Pipe 5'],
   'branch_consumption': 800.0,
   'tested': False},
  'Branch 2': {'pipes': ['Pipe 6', 'Pipe 7'],
   'branch_consumption': 800.0,
   'tested': False}}}

In [74]:
#start solve the network: begin with pump data:
###  Pump data  ###
pump_number = pumps_table['pump #'][0]
pump_flow = pipes_table['consumption'].sum()  
pumps_table['flow'][pump_number] = pump_flow
pump1 = entities.Pump(rated_flow=pump_flow)

previous_pipe_number = 0

for pipe in range(0,len(pipes_table.index)):

    branch_index = 1
    if pipe == 0:
        new_pipe_number = pipes_table[pipes_table['start with'] == pump_number]['end with'].index[0]
        flow_rate = pump_flow

    elif (previous_pipe_number in junctions_list) or (pipes_table['end with'][previous_pipe_number] == ""):
        found_not_tested_branch = False
        # Iterate over the main dictionary
        for pipe, branches in branches_tree.items():
            # Iterate over the branches for each pipe
            for branch, branch_info in branches.items():
                # Check if the branch is not tested
                if not branch_info['tested']:
                    branch_info['tested'] = True
                    print(f"Branch '{branch}' under '{pipe}' is not tested.")
                    new_pipe_number = branch_info['pipes'][0]
                    found_not_tested_branch = True
                    break
            if found_not_tested_branch:
                break
        
            
        # new_pipe_number = branches[previous_pipe_number]['Branch 1']['pipes'][0] 
        flow_rate = pipes_table['flow'][new_pipe_number]

    else:
        new_pipe_number = pipes_table['end with'][previous_pipe_number]
        flow_in = float(pipes_table['flow'][previous_pipe_number])
        consumption = float(pipes_table['consumption'][previous_pipe_number])
        flow_rate = flow_in - consumption
        
    pipetype = pipes_table['pipe type'][new_pipe_number]
    nominal_dia = pipes_table['nominal dia'][new_pipe_number]
    inside_dia = pipes_table['id (mm)'][new_pipe_number]
    length = pipes_table['length (m)'][new_pipe_number]
    static_head = pipes_table['static head (endZ-startZ)'][new_pipe_number]

    pipe1 = entities.Pipe(pipetype=pipetype,nominal_dia=nominal_dia,inside_dia=inside_dia,length=length,static_head=static_head,flow_rate=flow_rate)

    velocity = pipe1.velocity()
    head_loss = pipe1.major_head_loss()
    total_headloss = pipe1.total_head_loss()

    pipes_table['flow'][new_pipe_number] = flow_rate 
    pipes_table['velocity'][new_pipe_number] = velocity
    pipes_table['head loss'][new_pipe_number] = head_loss
    pipes_table['total head loss'][new_pipe_number] = total_headloss

    previous_pipe_number = new_pipe_number

# minimum_pressure = add_pressure_at_end_of_pipe(pipes_table)



C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pump_number = pumps_table['pump #'][0]
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pumps_table['flow'][pump_number] = pump_flow
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

Branch 'Branch 1' under 'Pipe 3' is not tested.


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][new_pipe_number] = flow_rate
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['velocity'][new_pipe_number] = velocity
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

Branch 'Branch 2' under 'Pipe 3' is not tested.


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][new_pipe_number] = flow_rate
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['velocity'][new_pipe_number] = velocity
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\3131714732.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [63]:
for pipe, branches in branches_tree.items():
            # Iterate over the branches for each pipe
            for branch, branch_info in branches.items():
                # Check if the branch is not tested
                if not branch_info['tested']:
                    print(f"Branch '{branch}' under '{pipe}' is not tested.")

Branch 'Branch 1' under 'Pipe 3' is not tested.
Branch 'Branch 2' under 'Pipe 3' is not tested.


In [87]:
def are_branches_not_tested():
        found_not_tested_branch = False
        # Iterate over the main dictionary
        for pipe, branches in branches_tree.items():
            # Iterate over the branches for each pipe
            for branch, branch_info in branches.items():
                # Check if the branch is not tested
                if not branch_info['tested']:
                    branch_info['tested'] = True
                    print(f"Branch '{branch}' under '{pipe}' is not tested.")
                    return branch_info['pipes'][0]

#start solve the network: begin with pump data:
###  Pump data  ###
pump_number = pumps_table['pump #'][0]
pump_flow = pipes_table['consumption'].sum()  
pumps_table['flow'][pump_number] = pump_flow
pump1 = entities.Pump(rated_flow=pump_flow)

previous_pipe_number = 0

for pipe in range(0,len(pipes_table.index)):

    branch_index = 1
    if pipe == 0:
        new_pipe_number = pipes_table[pipes_table['start with'] == pump_number]['end with'].index[0]
        flow_rate = pump_flow

    elif (previous_pipe_number in junctions_list) or (pipes_table['end with'][previous_pipe_number] == ""):
        
        new_pipe_number = are_branches_not_tested()
        # found_not_tested_branch = False
        # # Iterate over the main dictionary
        # for pipe, branches in branches_tree.items():
        #     # Iterate over the branches for each pipe
        #     for branch, branch_info in branches.items():
        #         # Check if the branch is not tested
        #         if not branch_info['tested']:
        #             branch_info['tested'] = True
        #             print(f"Branch '{branch}' under '{pipe}' is not tested.")
        #             new_pipe_number = branch_info['pipes'][0]
        #             found_not_tested_branch = True
        #             break
        #     if found_not_tested_branch:
        #         break
        
            
        # new_pipe_number = branches[previous_pipe_number]['Branch 1']['pipes'][0] 
        flow_rate = pipes_table['flow'][new_pipe_number]

    else:
        new_pipe_number = pipes_table['end with'][previous_pipe_number]
        flow_in = float(pipes_table['flow'][previous_pipe_number])
        consumption = float(pipes_table['consumption'][previous_pipe_number])
        flow_rate = flow_in - consumption
        
    pipetype = pipes_table['pipe type'][new_pipe_number]
    nominal_dia = pipes_table['nominal dia'][new_pipe_number]
    inside_dia = pipes_table['id (mm)'][new_pipe_number]
    length = pipes_table['length (m)'][new_pipe_number]
    static_head = pipes_table['static head (endZ-startZ)'][new_pipe_number]

    pipe1 = entities.Pipe(pipetype=pipetype,nominal_dia=nominal_dia,inside_dia=inside_dia,length=length,static_head=static_head,flow_rate=flow_rate)

    velocity = pipe1.velocity()
    head_loss = pipe1.major_head_loss()
    total_headloss = pipe1.total_head_loss()

    pipes_table['flow'][new_pipe_number] = flow_rate 
    pipes_table['velocity'][new_pipe_number] = velocity
    pipes_table['head loss'][new_pipe_number] = head_loss
    pipes_table['total head loss'][new_pipe_number] = total_headloss

    previous_pipe_number = new_pipe_number

# minimum_pressure = add_pressure_at_end_of_pipe(pipes_table)


print(are_branches_not_tested())

C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pump_number = pumps_table['pump #'][0]
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pumps_table['flow'][pump_number] = pump_flow
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

Branch 'Branch 1' under 'Pipe 3' is not tested.


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][new_pipe_number] = flow_rate
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['velocity'][new_pipe_number] = velocity
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

Branch 'Branch 2' under 'Pipe 3' is not tested.


C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['flow'][new_pipe_number] = flow_rate
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pipes_table['velocity'][new_pipe_number] = velocity
C:\Users\Avner.PALGEY-MAIM\AppData\Local\Temp\ipykernel_33404\1571903523.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

None


In [86]:
branches_tree

{'Pipe 3': {'Branch 1': {'pipes': ['Pipe 4', 'Pipe 5'],
   'branch_consumption': 800.0,
   'tested': False},
  'Branch 2': {'pipes': ['Pipe 6', 'Pipe 7'],
   'branch_consumption': 800.0,
   'tested': False}}}

In [75]:
with pd.ExcelWriter('data\\outputs\\acad-pipelines.xlsx') as writer:
        pipes_table.to_excel(writer,sheet_name='Pipes',index=False)
        pumps_table.to_excel(writer,sheet_name='Pumps',index=False)
        channels_table.to_excel(writer,sheet_name='Channels',index=False)    
os.startfile('data\\outputs\\acad-pipelines.xlsx') 